# Lesson-1

# Linear Classification and Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
#Loading up a dataset
df = pd.read_csv('NFA2019public_data.csv')

C:\Users\nblsh\AppData\Local\Temp\ipykernel_21208\1062772942.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('NFA2019public_data.csv')


In [3]:
#info
df.info('QScore')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72186 entries, 0 to 72185
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         72186 non-null  object 
 1   year            72186 non-null  int64  
 2   country_code    72186 non-null  int64  
 3   record          72186 non-null  object 
 4   crop_land       51714 non-null  float64
 5   grazing_land    51714 non-null  float64
 6   forest_land     51714 non-null  object 
 7   fishing_ground  51713 non-null  float64
 8   built_up_land   51713 non-null  float64
 9   carbon          51713 non-null  float64
 10  total           72177 non-null  float64
 11  QScore          72185 non-null  object 
dtypes: float64(6), int64(2), object(4)
memory usage: 6.6+ MB


In [4]:
df[ 'QScore' ].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [5]:
df.isna().sum() 

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [6]:
#dropping null values
df = df.dropna() 

In [7]:
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [8]:
df[ 'QScore' ] = df[ 'QScore' ].replace([ '1A' ], '2A' )

In [9]:
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [10]:
df_2A = df[df.QScore== '2A' ]

In [11]:
df_3A = df[df.QScore== '3A' ].sample( 350 )

In [12]:
data_df = df_2A.append(df_3A)

C:\Users\nblsh\AppData\Local\Temp\ipykernel_21208\1154166429.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = df_2A.append(df_3A)


In [13]:
#processing
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop= True )
data_df.shape
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [14]:
#more processing
data_df = data_df.drop(columns=[ 'country_code' , 'country' , 'year' ])
X = data_df.drop(columns= 'QScore' )
y = data_df[ 'QScore' ] 


In [15]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 , random_state= 0 )
y_train.value_counts() 

3A    250
2A    163
Name: QScore, dtype: int64

In [16]:
#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)

In [17]:
x_test.record = encoder.transform(x_test.record)

In [18]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state= 1 )

In [19]:
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)

In [20]:
x_train_balanced = pd.DataFrame(x_train_balanced, columns=x_train.columns)

In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df,columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df,
columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [22]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression()

# Lesson-2

# Measuring Classification Performance

## Cross-validation and accuracy

In [24]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv= 5 , scoring= 'f1_macro' )
scores 

array([0.55716586, 0.52768566, 0.6       , 0.549955  , 0.57729469])

## Confusion Matrix

In [23]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A', '3A'])
cnf_mat

array([[39, 38],
       [50, 50]], dtype=int64)

## K-Fold Cross Validation 

In [29]:
from sklearn.model_selection import KFold
kf = KFold(n_splits= 5 )
kf.split(normalised_train_df)
f1_scores = []
#run for every split
for train_index, test_index in kf.split(normalised_train_df):
    
    x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
    
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]
    
    model = LogisticRegression().fit(x_train, y_train)
    #save result to list
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test),
                              pos_label= '2A' )* 100)

## Accuracy

In [25]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 )) 

Accuracy: 50


## Precision

In [26]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 44


## Recall

In [27]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'Recall: {}' .format(round(recall* 100 ), 2 ))

Recall: 51


## F-1 Score

In [28]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 47


## Stratified K-Fold

In [30]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits= 5 , shuffle= True , random_state= 1 )
f1_scores = []
#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    
    x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]
    
    model = LogisticRegression().fit(x_train, y_train)
    
    #save result to list
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label= '2A' ))

## Leave One Out Cross Validation (LOOCV) 

In [31]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo,
 scoring= 'f1_macro' )
average_score = scores.mean() * 100 

In [32]:
average_score

56.599999999999994

In [33]:
f1_scores

[0.5833333333333334,
 0.4946236559139785,
 0.5714285714285713,
 0.5510204081632654,
 0.58]

# Lesson-4

## Decision Tree

In [34]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier()

# Lesson-5

## Bagging

In [35]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
# fit the model on the whole dataset
model.fit(normalised_train_df, y_balanced)

RandomForestClassifier()

## Boosting

In [36]:
from sklearn.ensemble import AdaBoostClassifier
model1 = AdaBoostClassifier()
# fit the model on the whole dataset
model1.fit(normalised_train_df, y_balanced)

AdaBoostClassifier()

In [37]:
from sklearn.ensemble import GradientBoostingClassifier
model2 = GradientBoostingClassifier()
# fit the model on the whole dataset
model2.fit(normalised_train_df, y_balanced)

GradientBoostingClassifier()